# 26th April Notebook

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import seaborn as sns

## Making data for x + y = z

In [47]:
data = torch.randint(-10000, 10000, (2000, 2)).float()
data

tensor([[ 2188., -7775.],
        [ 5605., -2786.],
        [-9826.,  3784.],
        ...,
        [ -211.,  3639.],
        [-4849.,   440.],
        [ 6474.,  4820.]])

In [35]:
labels = data[:, 0] + data[:, 1]

In [64]:
labels = labels.float()
labels

tensor([ -8199., -11151.,   7934.,  ...,   9369., -12998.,  -5122.])

## making data ready

In [65]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels,
                                                                   test_size=0.2)

In [66]:
train_data = TensorDataset(train_data, train_labels)
test_data = TensorDataset(test_data, test_labels)

In [67]:
train_loader = DataLoader(train_data,
                         batch_size=32,
                         shuffle=True,
                         drop_last=True)
test_loader = DataLoader(test_data,
                        batch_size=test_data.tensors[0].shape[0])

## Making model

In [68]:
def getModel():
    
    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            # input layer
            self.input = nn.Linear(2, 32)
            self.hidden_1 = nn.Linear(32, 1)
            # output layer
            self.output = nn.Linear(1, 1)
        
        # forward pass
        def forward(self, x):
            x = F.relu(self.input(x))
            x = F.relu(self.hidden_1(x))
            return self.output(x)
    
    Loss = nn.MSELoss()
    model = Model()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
    return model, optimizer, Loss

In [79]:
def train_model(epochs=5):
    model, optimizer, Loss = getModel()
    train_loss = []
    test_loss = []
    for i in range(epochs):
        batch_loss = []
        for x, y in train_loader:
            yh = model(x)
            loss = Loss(yh, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.item())
        train_loss.append(np.mean(batch_loss))
        print("MSE: ", train_loss[-1])
    return train_loss, test_loss, model


In [80]:
train_loss, test_loss, model = train_model()

MSE:  66472665.04
MSE:  66422242.48
MSE:  66444285.8
MSE:  66407563.36
MSE:  66413531.68


In [85]:
model.eval()
yh = model(torch.tensor([1., 2.]))

# Working with Heart disease dataset

In [168]:
data = pd.read_csv("data/heart.csv")
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [169]:
labels = data["target"]
data = data.drop("target", axis=1)
data = torch.tensor(data.values).float()
label = torch.tensor(labels.values).float()

In [171]:
data.shape, label.shape

(torch.Size([1025, 13]), torch.Size([1025]))

In [172]:
label = label[:, None]
label

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]])

## Data preprocssing

In [173]:
train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.2)

In [174]:
train_data = TensorDataset(train_data, train_label)
test_data = TensorDataset(test_data, test_label)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

## Preparing model

In [211]:
def getModel():
    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.input = nn.Linear(13, 32)
            self.hidden_1 = nn.Linear(32, 128)
            self.hidden_2 = nn.Linear(128, 64)
            self.hidden_2_2 = nn.Linear(64, 64)
            self.hidden_3 = nn.Linear(64, 32)
            self.output_layer = nn.Linear(32, 1)
        def forward(self, x):
            x = F.relu(self.input(x))
            x = F.relu(self.hidden_1(x))
            x = F.relu(self.hidden_2(x))
            x = F.relu(self.hidden_2_2(x))
            x = F.relu(self.hidden_3(x))
            return self.output_layer(x)
    
    model = Model()
    Loss = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    return model, Loss, optimizer

In [212]:
def train_model(epochs=50):
    train_acc = []
    train_loss = []
    test_acc = []
    model, Loss, optimizer = getModel()
    
    for i in range(epochs):
        batch_acc = []
        batch_loss = []
        for x, y in train_loader:
            yh = model(x)
            loss = Loss(yh, y)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            correct = (yh > .5) == y
            batch_acc.append(100 * torch.mean(correct.float()))
        train_acc.append(np.mean(batch_acc))
        # computing test
        x, y = next(iter(test_loader))
        with torch.no_grad():
            yh = model(x)
        test_acc.append(100 * torch.mean(((yh > .5) == y).float()))
        print(f"Train accuracy: {train_acc[-1]}, Test accuracy: {test_acc[-1]}")
    return train_acc, test_acc, model

In [213]:
train_acc, test_acc, model = train_model()

Train accuracy: 52.25, Test accuracy: 70.73170471191406
Train accuracy: 66.875, Test accuracy: 71.70731353759766
Train accuracy: 68.25, Test accuracy: 78.04877471923828
Train accuracy: 69.875, Test accuracy: 74.63414764404297
Train accuracy: 72.0, Test accuracy: 80.9756088256836
Train accuracy: 75.625, Test accuracy: 77.07316589355469
Train accuracy: 77.0, Test accuracy: 76.0975570678711
Train accuracy: 77.875, Test accuracy: 77.56097412109375
Train accuracy: 77.375, Test accuracy: 77.07316589355469
Train accuracy: 78.75, Test accuracy: 84.39024353027344
Train accuracy: 78.0, Test accuracy: 77.56097412109375
Train accuracy: 78.625, Test accuracy: 83.90243530273438
Train accuracy: 80.875, Test accuracy: 80.9756088256836
Train accuracy: 78.375, Test accuracy: 85.36585235595703
Train accuracy: 78.625, Test accuracy: 84.8780517578125
Train accuracy: 77.25, Test accuracy: 82.43902587890625
Train accuracy: 80.0, Test accuracy: 83.90243530273438
Train accuracy: 79.5, Test accuracy: 81.9512176